<a href="https://colab.research.google.com/github/satoruk-icepp/mlhep2019_2_phase/blob/master/Bayesian_lhcb_calo_gan_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')

KeyboardInterrupt: ignored

In [0]:
! wget https://raw.githubusercontent.com/SchattenGenie/mlhep2019_2_phase/master/analysis/calogan_metrics.py -nc
! wget https://raw.githubusercontent.com/SchattenGenie/mlhep2019_2_phase/master/analysis/prd_score.py -nc
! wget https://raw.githubusercontent.com/SchattenGenie/mlhep2019_2_phase/master/analysis/score.py -nc
! wget https://github.com/satoruk-icepp/mlhep2019_2_phase/raw/master/analysis/embedder.tp -nc
! wget https://github.com/satoruk-icepp/mlhep2019_2_phase/raw/master/generator.py -P /content/ -O generator.py
! wget https://github.com/satoruk-icepp/mlhep2019_2_phase/raw/master/discriminator.py -P /content/ -O discriminator.py
! wget https://github.com/satoruk-icepp/mlhep2019_2_phase/raw/master/analysis/Normalization.py -P /content/ -O Normalization.py
! wget https://github.com/satoruk-icepp/mlhep2019_2_phase/raw/master/analysis/bayes.py -P /content/ -O bayes.py
! wget https://github.com/satoruk-icepp/mlhep2019_2_phase/raw/master/Label2Image.py -P /content/ -O Label2Image.py
! wget https://github.com/satoruk-icepp/mlhep2019_2_phase/raw/master/Regressor.py -P /content/ -O Regressor.py
! wget https://github.com/satoruk-icepp/mlhep2019_2_phase/raw/master/NetworkUtil.py -P /content/ -O NetworkUtil.py

In [0]:
%%writefile .comet.config
[comet]
api_key=mIel5ZAPOioTs0Cij75dSSQXs
logging_file = /tmp/comet.log
logging_file_level = info


In [0]:
! [ ! -z "$COLAB_GPU" ] && pip install skorch comet_ml

In [0]:
BATCH_SIZE         = 150
N_EPOCH            = 200

learning_rate      = 0.00005
learning_rate_reg      = 0.001
dnoise_alpha       = 0.001
gnoise_alpha       = 0.001
N=50250
Nval = 1500
NOISEIMAGE_DIM = 10
NOISE_DIM = NOISEIMAGE_DIM**2
Ngen = 1
Ndisc = 1
EnergyDepositScale = 4000
EnergyDepositOffset = 0
PXscale = 30
PYscale = 30
PZscale = 50
PZoffset=50
XPosscale = 10
YPosscale = 10
PDGscale=11
PDGoffset=16.5
Nresblock = 10
CONDITION_DIM=6
IMAGE_DIM=30
dropout_disc = 0.00
unrolled_steps= 0
Nredconv_gen = 10
ntrain_d = 1
lambda_reg =10
# TASK = 'BAYESIAN'
TASK = 'WASSERSTEIN'
# TASK = 'CRAMER'
optimizer = 'Adam'
# optimizer = 'RMSProp'
# optimizer = 'RMSAdam'
weight_decay = 0e-05
add_dense = False
use_additionalinfo = False
restartfrom=0
do_reg = False
use_resblock = True
params={'batch_size' : BATCH_SIZE,
        'data_size'  : N,
        'epochs'     : N_EPOCH,
        'energyscale': EnergyDepositScale,
        'noise_dim'   : NOISE_DIM,
        'learning_rate':learning_rate,
        'dnoise_alpha':dnoise_alpha,
        'gnoise_alpha':gnoise_alpha,
        'Ngen':Ngen,
        'Ndisc':Ndisc,
        'PXscale':PXscale,
        'PYscale':PYscale,
        'PZscale':PZscale,
        'XPosscale':XPosscale,
        'YPosscale':YPosscale,
        'Nresblock':Nresblock,
        'dropout_disc':dropout_disc,
        'unrolled_steps':unrolled_steps,
        'ntrain_d':ntrain_d,
        'lambda_reg':lambda_reg,
        'task:':TASK,
        'optim':optimizer,
        'weight_decay':weight_decay,
        'Nredconv_gen':Nredconv_gen,
        'add_dense':add_dense,
        'use_additionalinfo':use_additionalinfo,
        'restartfrom':restartfrom,
        'Do_reg':do_reg,
        'use_resblock':use_resblock
}


In [0]:
from comet_ml import Experiment
experiment = Experiment(project_name="BayesGAN")
experiment.log_parameters(params)

In [0]:
from skorch import NeuralNetClassifier

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as utils
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
from torchvision import datasets, transforms
import time
import numpy as np
import math
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm, tqdm_notebook
import seaborn as sns
from IPython.display import clear_output
sns.set()

def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [0]:
device

## Data pathes

In [0]:
train_data_path = '/gdrive/My Drive/mlhep2019_gan/data_train.npz'
# train_data_path = '/gdrive/My Drive/data_afteraugmentation.npz'
val_data_path = '/gdrive/My Drive/mlhep2019_gan/data_val.npz'
test_data_path = '/gdrive/My Drive/mlhep2019_gan/data_test.npz'

# train_data_path = '../data_train.npz'
# val_data_path = '../data_val.npz'
# test_data_path = '../data_test.npz'

In [0]:
def GetDiscPath(epoch):
    return '/gdrive/My Drive/disc_%d.pt'%(epoch)
def GetGenPath(epoch):
    return '/gdrive/My Drive/gen_%d.pt'%(epoch)

# Loading data

Data is stored in `.npz`-format which is a special filetype for persisting multiple NumPy arrays on disk. 

More info: https://docs.scipy.org/doc/numpy/reference/generated/numpy.lib.format.html#module-numpy.lib.format.

File `dat_train.npz` contains four arrays: 

  * `EnergyDeposit` - images of calorimeters responses
  * `ParticleMomentum` - $p_x, p_y, p_z$ of initial partice
  * `ParticlePoint` - $x, y$ of initial particle
  * `ParticlePDG` - particle type(either $e^-$ or $\gamma$)

In [0]:
EnergyDepositScale = torch.tensor([EnergyDepositScale]).float()
EnergyDepositOffset = torch.tensor([EnergyDepositOffset]).float()
MomentumScale      = torch.tensor([PXscale,PYscale,PZscale]).float()
MomentumOffset      = torch.tensor([0,0,PZoffset]).float()
PointScale         = torch.tensor([XPosscale,YPosscale]).float()
PointOffset        = torch.tensor([0,0]).float()
ParticlePDGScale   = torch.tensor([PDGscale]).float()
ParticlePDGOffset   = torch.tensor([PDGoffset]).float()
MomentumPointPDGScale = torch.cat([MomentumScale,PointScale,ParticlePDGScale])
MomentumPointPDGOffset = torch.cat([MomentumOffset,PointOffset,ParticlePDGOffset])

In [0]:
# N = 1000

data_train = np.load(train_data_path, allow_pickle=True)
print(list(data_train.keys()))
N = len(data_train['EnergyDeposit'])
print(N)
# N=1500
# [data_size, 900]
EnergyDeposit = data_train['EnergyDeposit'][:N]
# reshaping it as [data_size, channels, img_size_x, img_size_y]
# channels are needed for pytorch conv2d-layers
EnergyDeposit = EnergyDeposit.reshape(-1,1,30,30)
# EnergyDeposit = EnergyDeposit/EnergyDepositScale

# [data_size, 3]
# ParticleMomentumScale = [PXscale,PYscale,PZscale]
ParticleMomentum = data_train['ParticleMomentum'][:N]
# ParticleMomentum = np.divide(ParticleMomentum,ParticleMomentumScale) 

# [data_size, 2]
# ParticlePointScale = [XPosscale,YPosscale]
ParticlePoint = data_train['ParticlePoint'][:, :2][:N]
# ParticlePoint = np.divide(ParticlePoint,ParticlePointScale)

# [data_size, 1]
ParticlePDG = data_train['ParticlePDG'][:N]
ParticlePDG = ParticlePDG.reshape(-1,1)

In [0]:
EnergyDeposit    = torch.tensor(EnergyDeposit).float()
ParticleMomentum = torch.tensor(ParticleMomentum).float()
ParticlePoint    = torch.tensor(ParticlePoint).float()
ParticlePDG      = torch.tensor(ParticlePDG).float()

In [0]:
# EnergyDeposit_1d = EnergyDeposit.reshape(-1,900)
# MaxElement = torch.argmax(EnergyDeposit_1d,1)
# SumElement = torch.sum(EnergyDeposit_1d,1)
# plt.hist(SumElement,range=(0,1000),bins=100)
# # len(SumElement)
# SumElement.shape

In [0]:

# EnergyDeposit = EnergyDeposit.reshape(-1,1,30,30)
# XProj = EnergyDeposit.sum(dim=2)
# YProj = EnergyDeposit.sum(dim=3)
# plt.imshow(EnergyDeposit[0][0])


# XProj = XProj.view(len(XProj),-1)
# print(XProj[0])
# # for i in range(len(XProj)):
# XMean = torch.zeros(len(XProj))
# XVar = torch.zeros(len(XProj))
# for i in range(1):
#     # XMean = 0
#     # XVar = 0
#     for j in range(len(XProj[i])):
#         XMean[i] += float(j)*XProj[i][j]/XProj[i].sum(dim=0)
#     for j in range(len(XProj[i])):
#         XVar[i] += ((j-XMean[i])*XProj[i][j])**2/XProj[i].sum(dim=0)
# print(XMean,XVar)
# YProj = YProj.view(len(YProj),-1)
# YMean = 0
# YVar = 0
# # for i in range(len(YProj)):
# #     YMean += float(i)*YProj[i]/YProj.sum(dim=0)
# # for i in range(len(YProj)):
# #     YVar += ((i-YMean)*YProj[i])**2
# print(YMean,YVar)
# MaxElement[0]//30
# MaxElement[0]%30
# torch.argmax(XProj).float()/30
# # len(YProj)

# YProj = torch.argmax(YProj,dim=1).float()
# YProj.shape
# # plt.hist(YProj,bins=[i for i in range(30)],range=(0,30))
# # plt.show()
EnergyDeposit_1d = EnergyDeposit.view(EnergyDeposit.shape[0],900)
# MaxElement = torch.argmax(EnergyDeposit_1d,dim=1)
SumElement = torch.sum(EnergyDeposit_1d,dim=1)
SumElement = SumElement.view(EnergyDeposit.shape[0],1)
## project image and extract mean and variance
XProj = EnergyDeposit.sum(dim=2).view(EnergyDeposit.shape[0],-1)
YProj = EnergyDeposit.sum(dim=3).view(EnergyDeposit.shape[0],-1)
XMean = torch.zeros(EnergyDeposit.shape[0],1)
XVar  = torch.zeros(EnergyDeposit.shape[0],1)
YMean = torch.zeros(EnergyDeposit.shape[0],1)
YVar  = torch.zeros(EnergyDeposit.shape[0],1)
indices = torch.tensor([float(i) for i in range(EnergyDeposit.shape[2])])
print(indices)
for i in range(5):
    XMean[i] = ((indices * XProj[i]).sum()/SumElement[i]-14.5)/15.0
    XVar[i]  = XProj[i].norm()/SumElement[i]
    
    # XMean[i] = (XMean[i]/SumElement[i]-14.5)/15.0
    # XVar[i] = torch.sqrt(XVar[i]/SumElement[i])

for i in range(5):
    YMean[i] = ((indices * YProj[i]).sum()/SumElement[i]-14.5)/15.0
    YVar[i]  = YProj[i].norm()/SumElement[i]
SumElement = SumElement/20000
AdditionalProperties = torch.cat([SumElement,XMean,XVar,YMean,YVar],dim=1)
print(AdditionalProperties)

In [0]:
# EnergyDeposit_check = torch.div(EnergyDeposit-EnergyDepositOffset,EnergyDepositScale)
# ParticleMomentum_ParticlePoint_ParticlePDG_check = torch.cat([ParticleMomentum, ParticlePoint,ParticlePDG], dim=1)
# ParticleMomentum_ParticlePoint_ParticlePDG_check = torch.div(ParticleMomentum_ParticlePoint_ParticlePDG_check-MomentumPointPDGOffset,MomentumPointPDGScale)
# # plt.hist(ParticleMomentum_ParticlePoint_ParticlePDG_check[:,4])

# # plt.hist(EnergyDeposit_check.reshape(EnergyDeposit_check.shape[0],-1))

## Load it to pytorch `DataLoader`

In [0]:


# validation_split = 0.05
random_seed= 42
split = Nval
indices = list(range(N))
np.random.seed(random_seed)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

calo_dataset    = utils.TensorDataset(EnergyDeposit, ParticleMomentum, ParticlePoint, ParticlePDG)

train_loader = torch.utils.data.DataLoader(calo_dataset, batch_size=BATCH_SIZE, 
                                           sampler=train_sampler, pin_memory=True)
validation_loader = torch.utils.data.DataLoader(calo_dataset, batch_size=BATCH_SIZE,
                                                sampler=valid_sampler, pin_memory=True)


# calo_dataloader = torch.utils.data.DataLoader(calo_dataset, batch_size=BATCH_SIZE, pin_memory=True, shuffle=True)

In [0]:
for EnergyDeposit_b, ParticleMomentum_b, ParticlePoint_b, ParticlePDG_b in validation_loader:
    break

## Training GAN
###### ...is not a simple matter

It depends on architecture, loss, instance noise, augmentation and even luck(recommend to take a look https://arxiv.org/pdf/1801.04406.pdf)


In this notebook I have prepared some basic parts that you could use for your experiments.

#### Small hack that can speed-up training and improve generalization

In [0]:
from bayes import NoiseLoss,PriorLoss

In [0]:
def add_instance_noise(data, std=0.01):
    return data + torch.distributions.Normal(0, std).sample(data.shape).to(device)

In [0]:
# class GANLosses(object):
#     def __init__(self, task, device):
#         self.TASK = task
#         self.device = device
def critic(disc_1,disc_2):
    return (torch.norm(disc_1 - disc_2,p=2,dim=-1) - torch.norm(disc_1,p=2,dim=-1)).mean()

def gen_loss_func(disc_real,disc_gen_1,disc_gen_2):
    return (torch.norm(disc_real - disc_gen_1,p=2,dim=-1) + 
            torch.norm(disc_real - disc_gen_2,p=2,dim=-1) -
            torch.norm(disc_gen_1 - disc_gen_2,p=2,dim=-1)
           ).mean()

def calc_gradient_penalty(discriminator, data_gen, inputs_batch, inp_data, lambda_reg = .1):
        alpha = torch.rand(inp_data.shape[0], 1).to(device)
        dims_to_add = len(inp_data.size()) - 2
        for i in range(dims_to_add):
            alpha = alpha.unsqueeze(-1)
        # alpha = alpha.expand(inp_data.size())

        interpolates = (alpha * inp_data + ((1 - alpha) * data_gen)).to(device)

        interpolates.requires_grad = True

        disc_interpolates,_,_ = discriminator(interpolates, inputs_batch)

        gradients = torch.autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                                        grad_outputs=torch.ones(disc_interpolates.size()).to(device),
                                        create_graph=True, retain_graph=True, only_inputs=True)[0]

        gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * lambda_reg
        return gradient_penalty

## Defining discriminator

## Defining generator

In [0]:
import Label2Image
from generator import ModelGConvTranspose
from discriminator import ModelD

## Check our models on one batch 

In [0]:

gens = []
discs = []

gen  = ModelGConvTranspose(
    z_dim=NOISEIMAGE_DIM, 
    MomentumPointPDGScale = MomentumPointPDGScale.to(device),
    MomentumPointPDGOffset=MomentumPointPDGOffset.to(device), 
    EnergyScale = EnergyDepositScale.to(device), 
    EnergyOffset = EnergyDepositOffset.to(device),
    Nredconv_gen=Nredconv_gen,
    add_dense = add_dense,
    use_resblock =use_resblock
    ).to(device)
# gen  = ModelGConvTranspose_SMALL(z_dim=NOISEIMAGE_DIM, MomentumPointPDGScale = MomentumPointPDGScale.to(device),EnergyScale = EnergyDepositScale.to(device), Nredconv_gen=Nredconv_gen).to(device)
use_bn = True
if TASK=='WASSERSTEIN' or TASK=='CRAMER':
    use_bn = False
    
disc = ModelD(
    cond_dim=6, 
    MomentumPointPDGScale=MomentumPointPDGScale.to(device), 
    MomentumPointPDGOffset=MomentumPointPDGOffset.to(device), 
    EnergyScale = EnergyDepositScale.to(device),
    EnergyOffset = EnergyDepositOffset.to(device),
    Nredconv_dis=Nresblock,
    dropout_fraction=dropout_disc,
    use_bn=use_bn,
    use_additionalinfo = use_additionalinfo
    ).to(device)

gen.weight_init(mean=0.0, std=0.02)
disc.weight_init(mean=0.0, std=0.02)

In [0]:
def LoadModels(epoch):
    gen.load_state_dict(torch.load(GetGenPath(epoch)))
    disc.load_state_dict(torch.load(GetDiscPath(epoch)))

In [0]:
if restartfrom>0:
    LoadModels(restartfrom)

In [0]:
EnergyDeposit_b, ParticleMomentum_b, ParticlePoint_b, ParticlePDG_b = EnergyDeposit_b.to(device), \
                                                       ParticleMomentum_b.to(device), \
                                                       ParticlePoint_b.to(device),\
                                                       ParticlePDG_b.to(device)
# print(ParticlePoint_b.shape,ParticlePDG_b.shape,)
# ParticlePDG_b= ParticlePDG_b.view(-1,1)
ParticleMomentum_ParticlePoint_ParticlePDG_b = torch.cat([ParticleMomentum_b.to(device), ParticlePoint_b.to(device),ParticlePDG_b.to(device)], dim=1)

In [0]:
# disc(EnergyDeposit_b, ParticleMomentum_ParticlePoint_ParticlePDG_b)

In [0]:
noise = torch.randn(len(EnergyDeposit_b), NOISEIMAGE_DIM, NOISEIMAGE_DIM).to(device)
gen(noise, ParticleMomentum_ParticlePoint_ParticlePDG_b).shape

In [0]:
reg_loss = torch.nn.SmoothL1Loss().to(device)
criterion = nn.BCELoss()
gprior_criterion = PriorLoss(prior_std=1., observed=N)
gnoise_criterion = NoiseLoss(params=gen.parameters(), noise_std=math.sqrt(2 * gnoise_alpha * learning_rate), observed=N)
dprior_criterion = PriorLoss(prior_std=1., observed=N)
dnoise_criterion = NoiseLoss(params=disc.parameters(), noise_std=math.sqrt(2 * dnoise_alpha * learning_rate), observed=N)

## Defining optimiser

In [0]:
if optimizer == "Adam":
    gen_opt  = optim.Adam(gen.parameters() , lr=learning_rate, betas=(0.9, 0.999), weight_decay=weight_decay)
    disc_opt = optim.Adam(disc.parameters(), lr=learning_rate, betas=(0.9, 0.999), weight_decay=weight_decay)
    reg_gen_opt = optim.Adam(gen.parameters(), lr=learning_rate_reg, betas=(0.9,0.999))
    reg_disc_opt = optim.Adam(disc.parameters(), lr=learning_rate_reg, betas=(0.9,0.999))
    
elif optimizer == "RMSProp":
    gen_opt  = optim.RMSprop(gen.parameters(), lr=learning_rate)
    disc_opt = optim.RMSprop(disc.parameters(), lr=learning_rate)
    reg_gen_opt = optim.RMSprop(gen.parameters(), lr=learning_rate_reg)
    reg_disc_opt = optim.RMSprop(disc.parameters(), lr=learning_rate_reg)
    
elif optimizer == "RMSAdam":
    gen_opt  = optim.Adam(gen.parameters() , lr=learning_rate, betas=(0.9, 0.999), weight_decay=weight_decay)
    disc_opt = optim.RMSprop(disc.parameters(), lr=learning_rate)    
    reg_gen_opt = optim.Adam(gen.parameters(), lr=learning_rate_reg, betas=(0.9,0.999))
    reg_disc_opt = optim.RMSprop(disc.parameters(), lr=learning_rate_reg)
    


## Load scores

In [0]:
from prd_score import compute_prd, compute_prd_from_embedding, _prd_to_f_beta
from sklearn.metrics import auc

In [0]:
from Regressor import Regressor, load_embedder
embedder = load_embedder('./embedder.tp')

In [0]:
def D_train(EnergyDeposit_b,ParticleMomentum_ParticlePoint_ParticlePDG_b,ibatch,gen_input=None):
    #=======================Train the discriminator=======================#
    
    disc_loss = 0
    disc_prior = 0
    disc_noise = 0
    disc_real = 0
    disc_fake = 0
    EnergyDeposit_b = add_instance_noise(EnergyDeposit_b)
    raw_real,output_real,mpp_real = disc(EnergyDeposit_b,ParticleMomentum_ParticlePoint_ParticlePDG_b)
    
    # disc_real = raw_real.mean()
    if gen_input is not None:
        EnergyDeposit_gen = gen_input
    else:
        noise = torch.randn(len(EnergyDeposit_b), NOISEIMAGE_DIM, NOISEIMAGE_DIM).to(device)
        EnergyDeposit_gen = gen(noise, ParticleMomentum_ParticlePoint_ParticlePDG_b)
    
    EnergyDeposit_gen = add_instance_noise(EnergyDeposit_gen)
    raw_fake,output_fake,mpp_fake = disc(EnergyDeposit_gen.detach(),ParticleMomentum_ParticlePoint_ParticlePDG_b)
    if do_reg == True:
        reg_loss_val = reg_loss(mpp_real, ParticleMomentum_ParticlePoint_ParticlePDG_b)
        reg_disc_opt.zero_grad()
        reg_loss_val.backward(retain_graph=True)
        reg_disc_opt.step()
        experiment.log_metric("d_reg", reg_loss_val.data.item(),step=ibatch)

    if TASK=="WASSERSTEIN":
        penalty = calc_gradient_penalty(disc,
                                        EnergyDeposit_gen.data,
                                        ParticleMomentum_ParticlePoint_ParticlePDG_b,
                                        EnergyDeposit_b.data,
                                        lambda_reg)
        disc_real = raw_real.mean()
        disc_fake = raw_fake.mean()
        disc_loss = disc_fake-disc_real+penalty
        experiment.log_metric("d_real", disc_real.data.item(),step=ibatch)
        experiment.log_metric("d_fake", disc_fake.data.item(),step=ibatch)
        experiment.log_metric("gradient_penalty", penalty.data.item(),step=ibatch)

    elif TASK == "BAYESIAN":
        labelv_real = Variable(0.9+0.1*torch.rand(len(EnergyDeposit_b), 1).to(device))
        labelv_fake = Variable(0.1*torch.rand(len(EnergyDeposit_b), 1).to(device)) 
        disc_real = criterion(output_real,labelv_real)
        disc_fake = criterion(output_fake, labelv_fake)
        disc_prior = dprior_criterion(disc.parameters())
        disc_noise = dnoise_criterion(disc.parameters())
        disc_loss = disc_real + disc_fake + disc_prior + disc_noise
        experiment.log_metric("d_real", disc_real.data.item(),step=ibatch)
        experiment.log_metric("d_fake", disc_fake.data.item(),step=ibatch)
        experiment.log_metric("d_prior", disc_prior.data.item(),step=ibatch)
        experiment.log_metric("d_noise", disc_noise.data.item(),step=ibatch)

    elif TASK == "CRAMER":
        noise_2 = torch.randn(len(EnergyDeposit_b), NOISEIMAGE_DIM, NOISEIMAGE_DIM).to(device)
        EnergyDeposit_gen_2 = gen(noise_2, ParticleMomentum_ParticlePoint_ParticlePDG_b)
        raw_fake_2,output_fake_2,mpp_fake_2 = disc(EnergyDeposit_gen_2.detach(),ParticleMomentum_ParticlePoint_ParticlePDG_b)
        penalty = calc_gradient_penalty(disc,
                                        EnergyDeposit_gen.data,
                                        ParticleMomentum_ParticlePoint_ParticlePDG_b,
                                        EnergyDeposit_b.data,
                                        lambda_reg)
        disc_loss = -critic(raw_real, raw_fake_2) + critic(raw_fake, raw_fake_2) + penalty
        experiment.log_metric("gradient_penalty", penalty.data.item(),step=ibatch)
    disc_opt.zero_grad()
    disc_loss.backward()
    disc_opt.step()
    print(disc_loss.data.item())
    
    experiment.log_metric("d_loss", disc_loss.data.item(),step=ibatch)
        
    return

In [0]:
def G_train(EnergyDeposit_b,ParticleMomentum_ParticlePoint_ParticlePDG_b,ibatch):
    #=======================Train the generator=======================#
    gen_opt.zero_grad()
    gen_loss = 0
    gen_real = 0
    gen_prior = 0
    gen_noise = 0
    noise = torch.randn(len(EnergyDeposit_b), NOISEIMAGE_DIM,NOISEIMAGE_DIM).to(device)
    EnergyDeposit_gen = gen(noise, ParticleMomentum_ParticlePoint_ParticlePDG_b)
    EnergyDeposit_gen = add_instance_noise(EnergyDeposit_gen)
    # if unrolled_steps>0:
    # disc_back = copy.deepcopy(disc)
    # for istep in range(unrolled_steps):
    #     D_train(EnergyDeposit_b,ParticleMomentum_ParticlePoint_ParticlePDG_b,EnergyDeposit_gen)
    
    raw_gen,output_gen,mpp_gen = disc(EnergyDeposit_gen,ParticleMomentum_ParticlePoint_ParticlePDG_b)
    if do_reg == True:
        reg_loss_val = reg_loss(mpp_gen, ParticleMomentum_ParticlePoint_ParticlePDG_b)
        reg_gen_opt.zero_grad()
        reg_loss_val.backward(retain_graph=True)
        reg_gen_opt.step()
        experiment.log_metric("g_reg", reg_loss_val.data.item(),step=ibatch)
    if TASK=="WASSERSTEIN":
        gen_real = raw_gen.mean()
        gen_loss = -gen_real
        experiment.log_metric("g_real_loss", gen_real.data.item(),step=ibatch)
    elif TASK=="BAYESIAN":
        labelv_real = Variable(torch.ones(len(EnergyDeposit_b), 1).to(device))
        gen_real = criterion(output_gen,labelv_real)
        gen_prior = gprior_criterion(gen.parameters())
        gen_noise = gnoise_criterion(gen.parameters())
        gen_loss = gen_real+gen_prior+gen_noise
        experiment.log_metric("g_real_loss", gen_real.data.item(),step=ibatch)
        experiment.log_metric("g_prior_loss", gen_prior.data.item(),step=ibatch)
        experiment.log_metric("g_noise_loss", gen_noise.data.item(),step=ibatch)
    elif TASK=="CRAMER":
        raw_real,output_real = disc(EnergyDeposit_b,ParticleMomentum_ParticlePoint_ParticlePDG_b)
        noise_2 = torch.randn(len(EnergyDeposit_b), NOISEIMAGE_DIM, NOISEIMAGE_DIM).to(device)
        EnergyDeposit_gen_2 = gen(noise_2, ParticleMomentum_ParticlePoint_ParticlePDG_b)
        raw_fake_2,output_fake_2,mpp_fake_2 = disc(EnergyDeposit_gen_2.detach(),ParticleMomentum_ParticlePoint_ParticlePDG_b)
        gen_loss = gen_loss_func(raw_real,raw_gen,raw_fake_2)
    # gen_real.backward()
        # gen_loss.data.item(),gen_real.data.item(),gen_prior.data.item(),gen_noise.data.item()
    experiment.log_metric("g_loss", gen_loss.data.item(),step=ibatch)
    
             
    gen_loss.backward()
    gen_opt.step()
    # disc.load(disc_back)
    return EnergyDeposit_gen

In [0]:
def CalculatePRDScore(EnergyDeposit_b,EnergyDeposit_gen):
    batch = 0
    plt.figure(figsize=(15,12))
    prd_aucs = []
    for EnergyDeposit_b, ParticleMomentum_b, ParticlePoint_b, ParticlePDG_b in validation_loader:
        noise = torch.randn(len(EnergyDeposit_b), NOISEIMAGE_DIM, NOISEIMAGE_DIM).to(device)
        ParticleMomentum_ParticlePoint_ParticlePDG_b = torch.cat([ParticleMomentum_b.to(device), ParticlePoint_b.to(device),ParticlePDG_b.to(device)], dim=1)
        EnergyDeposit_gen = gen(noise, ParticleMomentum_ParticlePoint_ParticlePDG_b)

        data_real = embedder.get_encoding(torch.tensor(EnergyDeposit_b).float().view(-1, 1, 30, 30)).detach().numpy()
        data_fake = embedder.get_encoding(torch.tensor(EnergyDeposit_gen.cpu()).float().view(-1, 1, 30, 30)).detach().numpy()
        precision, recall = compute_prd_from_embedding(data_real, data_fake,num_clusters=20,num_runs=10)
        # precisions, recalls = calc_pr_rec(data_real,data_fake)
        prd_auc_ = auc(precision, recall)
        prd_aucs.append(prd_auc_)
        plt.step(recall, precision, color='r', alpha=1,  label='PR')
        # experiment.log_figure(figure=plt)
        # break
        if batch>3:
            experiment.log_figure(figure=plt)
            break
        else:
            batch+=1
    np_prd_aucs = np.array(prd_aucs)
    prd_auc_ = np.mean(np_prd_aucs)
    return prd_auc_

In [0]:
def SaveModels(disc,gen,epoch):
    
    # gen  = ModelGConvTranspose_SMALL(z_dim=NOISEIMAGE_DIM, MomentumPointPDGScale = MomentumPointPDGScale.to(device),EnergyScale = EnergyDepositScale.to(device), Nredconv_gen=Nredconv_gen).to(device)
    use_bn = True
    if TASK=='WASSERSTEIN' or TASK=='CRAMER':
        use_bn = False
        
    disc_cpu = ModelD(
        cond_dim=6, 
        MomentumPointPDGScale=MomentumPointPDGScale.to(device), 
        MomentumPointPDGOffset=MomentumPointPDGOffset.to(device), 
        EnergyScale = EnergyDepositScale.to(device),
        EnergyOffset = EnergyDepositOffset.to(device),
        Nredconv_dis=Nresblock,
        dropout_fraction=dropout_disc,
        use_bn=use_bn,
        use_additionalinfo = use_additionalinfo
        )
    
    disc_cpu.load_state_dict(disc.state_dict())
    # generator_cpu.eval()
    torch.save(disc_cpu.state_dict(), GetDiscPath(epoch))
    gen_cpu  = ModelGConvTranspose(
        z_dim=NOISEIMAGE_DIM, 
        MomentumPointPDGScale = MomentumPointPDGScale.to(device),
        MomentumPointPDGOffset=MomentumPointPDGOffset.to(device), 
        EnergyScale = EnergyDepositScale.to(device), 
        EnergyOffset = EnergyDepositOffset.to(device),
        Nredconv_gen=Nredconv_gen,
        add_dense = add_dense,
        use_resblock =use_resblock
    )
    gen_cpu.load_state_dict(gen.state_dict())
    # generator_cpu.eval()
    torch.save(gen_cpu.state_dict(), GetGenPath(epoch))
    return

In [0]:
LIPSITZ_WEIGHTS = False
# ntrain_d = 1
def run_training(epochs):
    ibatch = 0
    dis_epoch_loss = []
    gen_epoch_loss = []
    predictions_dis = []
    predictions_gen = []
    TotalEnergyDeposit_dis = []
    TotalEnergyDeposit_gen = []    
    prd_auc = []  
    for epoch in tqdm(range(restartfrom,epochs)):
        first = True
        for EnergyDeposit_b, ParticleMomentum_b, ParticlePoint_b, ParticlePDG_b in train_loader:
            EnergyDeposit_b, ParticleMomentum_b, ParticlePoint_b, ParticlePDG_b = EnergyDeposit_b.to(device), \
                                                                   ParticleMomentum_b.to(device), \
                                                                   ParticlePoint_b.to(device),\
                                                                   ParticlePDG_b.view(-1,1).to(device)

            ParticleMomentum_ParticlePoint_ParticlePDG_b = torch.cat([ParticleMomentum_b.to(device), ParticlePoint_b.to(device), ParticlePDG_b.to(device)], dim=1)
            for itrain in range(ntrain_d):
                D_train(EnergyDeposit_b, ParticleMomentum_ParticlePoint_ParticlePDG_b,ibatch)
            EnergyDeposit_gen = G_train(EnergyDeposit_b, ParticleMomentum_ParticlePoint_ParticlePDG_b,ibatch)
            if LIPSITZ_WEIGHTS:                    
                [p.data.clamp_(-0.01, 0.01) for p in disc.parameters()]
            
            if first:
            # if ibatch %300 ==1:
                prd_auc_ = CalculatePRDScore(EnergyDeposit_b,EnergyDeposit_gen)
                prd_auc.append(prd_auc_)
                experiment.log_metric("PRD_AUC", prd_auc_,step=epoch)
                SaveModels(disc,gen,epoch)
                first=False
            
            clear_output()
            if ibatch %50 ==2:
                plt.figure(figsize=(15,12))
                grid = plt.GridSpec(2, 3, wspace=0.4, hspace=0.3)
                event = 0
                plt.subplot(grid[0,0])
                plt.imshow(EnergyDeposit_b[event][0].cpu().detach().numpy(),vmin=0,origin='lower')
                plt.title("Momentum: (%f,%f,%f) \n Position: (%f,%f)"%(
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][0],
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][1],
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][2],
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][3],
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][4]
                    )
                )
                # plt.semilogy(dis_epoch_loss, label='dis_epoch_loss')
                # plt.semilogy(gen_epoch_loss, label='gen_epoch_loss')
                # plt.legend()
                event = 1
                plt.subplot(grid[0,1])
                plt.imshow(EnergyDeposit_b[event][0].cpu().detach().numpy(),vmin=0,origin='lower')
                plt.title("Momentum: (%f,%f,%f)\n Position: (%f,%f)"%(
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][0],
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][1],
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][2],
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][3],
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][4]
                    )
                )
                # plt.hist(predictions_dis[-1], bins=40, label='dis_epoch_loss',color="blue",range=[0,1])
                # plt.hist(predictions_gen[-1], bins=40, label='gen_epoch_loss',color="orange",range=[0,1])
                # plt.legend()
                event = 2
                plt.subplot(grid[0,2])
                plt.imshow(EnergyDeposit_b[event][0].cpu().detach().numpy(),vmin=0,origin='lower')
                plt.title("Momentum: (%f,%f,%f)\n Position: (%f,%f)"%(
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][0],
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][1],
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][2],
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][3],
                    ParticleMomentum_ParticlePoint_ParticlePDG_b[event][4]
                    )
                )
                # plt.plot(prd_auc, label='prd_auc')
                # plt.ylim(0,1)
                # plt.legend()

                plt.subplot(grid[1, 0])
                plt.imshow(EnergyDeposit_gen[0][0].cpu().detach().numpy(),vmin=0,origin='lower')
                plt.subplot(grid[1, 1])
                plt.imshow(EnergyDeposit_gen[1][0].cpu().detach().numpy(),vmin=0,origin='lower')                
                plt.subplot(grid[1, 2])
                plt.imshow(EnergyDeposit_gen[2][0].cpu().detach().numpy(),vmin=0,origin='lower')                
                
                experiment.log_figure(figure_name="status%d"%(ibatch),figure=plt,step =ibatch)
                time.sleep(0.5)
#             plt.show()
            ibatch += 1



In [0]:
with experiment.train():
    run_training(N_EPOCH)

#### Transfer generator on CPU:

In [0]:
generator_cpu = ModelGConvTranspose(z_dim=NOISEIMAGE_DIM,MomentumPointPDGScale = MomentumPointPDGScale,EnergyScale = EnergyDepositScale)
# generator_cpu.load_state_dict(gen.state_dict())
generator_cpu.load_state_dict(torch.load('/gdrive/My Drive/gan80.pt'))
# generator_cpu.eval()

### Save model on disk

In [0]:
torch.save(generator_cpu.state_dict(), './gan_middle.pt')

## Making predictions

#### Validation predictions

In [0]:
data_val = np.load(val_data_path, allow_pickle=True)
ParticleMomentum_val = torch.tensor(data_val['ParticleMomentum']).float()
ParticlePoint_val = torch.tensor(data_val['ParticlePoint'][:, :2]).float()
ParticlePDG_val = torch.tensor(data_val['ParticlePDG'].reshape(-1,1)).float()
ParticleMomentum_ParticlePoint_ParticlePDG_val = torch.cat([ParticleMomentum_val, ParticlePoint_val,ParticlePDG_val], dim=1)
calo_dataset_val = utils.TensorDataset(ParticleMomentum_ParticlePoint_ParticlePDG_val)
calo_dataloader_val = torch.utils.data.DataLoader(calo_dataset_val, batch_size=1024, shuffle=False)

with torch.no_grad():
    EnergyDeposit_val = []
    for ParticleMomentum_ParticlePoint_ParticlePDG_val_batch in tqdm(calo_dataloader_val):
        noise = torch.randn(len(ParticleMomentum_ParticlePoint_ParticlePDG_val_batch[0]), NOISEIMAGE_DIM,NOISEIMAGE_DIM)
        EnergyDeposit_val_batch = generator_cpu(noise, ParticleMomentum_ParticlePoint_ParticlePDG_val_batch[0]).detach().numpy()
        EnergyDeposit_val.append(EnergyDeposit_val_batch)
    np.savez_compressed('./data_val_prediction.npz', 
                        EnergyDeposit=np.concatenate(EnergyDeposit_val, axis=0).reshape(-1, 30, 30))

    del EnergyDeposit_val
del data_val; del ParticleMomentum_val; del ParticlePoint_val; del ParticleMomentum_ParticlePoint_ParticlePDG_val;
del calo_dataset_val; calo_dataloader_val

#### Test predictions

In [0]:
data_test = np.load(test_data_path, allow_pickle=True)
ParticleMomentum_test = torch.tensor(data_test['ParticleMomentum']).float()
ParticlePoint_test = torch.tensor(data_test['ParticlePoint'][:, :2]).float()
ParticlePDG_test = torch.tensor(data_test['ParticlePDG'].reshape(-1,1)).float()
ParticleMomentum_ParticlePoint_ParticlePDG_test = torch.cat([ParticleMomentum_test, ParticlePoint_test, ParticlePDG_test], dim=1)
calo_dataset_test = utils.TensorDataset(ParticleMomentum_ParticlePoint_ParticlePDG_test)
calo_dataloader_test = torch.utils.data.DataLoader(calo_dataset_test, batch_size=1024, shuffle=False)

with torch.no_grad():
    EnergyDeposit_test = []
    for ParticleMomentum_ParticlePoint_ParticlePDG_test_batch in tqdm(calo_dataloader_test):
        noise = torch.randn(len(ParticleMomentum_ParticlePoint_ParticlePDG_test_batch[0]), NOISE_DIM)
        EnergyDeposit_test_batch = generator_cpu(noise, ParticleMomentum_ParticlePoint_ParticlePDG_test_batch[0]).detach().numpy()
        EnergyDeposit_test.append(EnergyDeposit_test_batch)
    np.savez_compressed('./data_test_prediction.npz', 
                        EnergyDeposit=np.concatenate(EnergyDeposit_test, axis=0).reshape(-1, 30, 30))

    del EnergyDeposit_test
del data_test; del ParticleMomentum_test; del ParticlePoint_test; del ParticleMomentum_ParticlePoint_ParticlePDG_test;
del calo_dataset_test; calo_dataloader_test

## `zip-zip` files together

In [0]:
!zip solution.zip data_val_prediction.npz data_test_prediction.npz

In [0]:
from IPython.display import FileLink
FileLink('./solution.zip')

# A few words about metrics

### Lets generate some fake data

In [0]:
for EnergyDeposit_b, ParticleMomentum_b, ParticlePoint_b, ParticlePDG_b in validation_loader:
    break

In [0]:
noise = torch.randn(len(ParticleMomentum_b), NOISE_DIM)
with torch.no_grad():
    ParticleMomentum_ParticlePoint_ParticlePDG = torch.cat([ParticleMomentum_b, 
                                                ParticlePoint_b,
                                                ParticlePDG_b], dim=1)
    EnergyDeposit_gen = generator_cpu(noise, ParticleMomentum_ParticlePoint_ParticlePDG.cpu())

In [0]:
EnergyDeposit_gen = EnergyDeposit_gen.detach().cpu().numpy().reshape(-1, 30, 30)
EnergyDeposit_b = EnergyDeposit_b.detach().cpu().numpy().reshape(-1, 30, 30)

#### Plot one image

In [0]:
plt.figure()
plt.imshow(EnergyDeposit_gen[2])
plt.colorbar()

## Calculate PRD score between these batch 

In [0]:
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        self.batchnorm0 = nn.BatchNorm2d(1)
        self.conv1 = nn.Conv2d(1, 16, 2, stride=2)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 2, stride=2)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 2, stride=2)
        self.batchnorm3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 2)
        
        self.dropout = nn.Dropout(p=0.3)
        
        self.fc1 = nn.Linear(256, 256) 
        self.batchnorm4 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 2 + 3)
        self.fc5 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.batchnorm0(self.dropout(x))
        x = self.batchnorm1(self.dropout(F.relu(self.conv1(x))))
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.batchnorm3(F.relu(self.conv3(x)))
        x = F.relu(self.conv4(x)) # 64, 5, 5
        x = x.view(len(x), -1)
        x = self.dropout(x)
        x = self.batchnorm4(self.dropout(F.relu(self.fc1(x))))
        x = F.leaky_relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        return self.fc4(x), self.fc5(x)
    
    def get_encoding(self, x):
        x = self.batchnorm0(self.dropout(x))
        x = self.batchnorm1(self.dropout(F.relu(self.conv1(x))))
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.batchnorm3(F.relu(self.conv3(x)))
        x = F.relu(self.conv4(x)) # 64, 5, 5
        x = x.view(len(x), -1)
        x = self.dropout(x)
        x = self.batchnorm4(self.dropout(F.relu(self.fc1(x))))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

def load_embedder(path):
    embedder = torch.load(path)
    embedder.eval()
    return embedder

embedder = load_embedder('./embedder.tp')

In [0]:
data_real = embedder.get_encoding(torch.tensor(EnergyDeposit_b).float().view(-1, 1, 30, 30)).detach().numpy()
data_fake = embedder.get_encoding(torch.tensor(EnergyDeposit_gen).float().view(-1, 1, 30, 30)).detach().numpy()

In [0]:
def plot_pr_aucs(precisions, recalls):
    plt.figure(figsize=(12, 12))
    pr_aucs = []
    for i in range(len(recalls)):
        plt.step(recalls[i], precisions[i], color='b', alpha=0.2,  label='PR-AUC={}'.format(auc(precisions[i], recalls[i])))
        pr_aucs.append(auc(precisions[i], recalls[i]))
    plt.step(np.mean(recalls, axis=0), np.mean(precisions, axis=0), color='r', alpha=1,  label='average')
    plt.fill_between(np.mean(recalls, axis=0), 
                     np.mean(precisions, axis=0) - np.std(precisions, axis=0) * 3,
                     np.mean(precisions, axis=0) + np.std(precisions, axis=0) * 3, color='g', alpha=0.2,  label='std')

    plt.xlabel('Recall')
    plt.ylabel('Precision')

    # plt.ylim([0.0, 1.05])
    # plt.xlim([0.0, 1.0])
    print(np.mean(pr_aucs), np.std(pr_aucs))
    plt.legend()
    
    return pr_aucs

def calc_pr_rec(data_real, data_fake, num_clusters=20, num_runs=10, NUM_RUNS=10):
    precisions = []
    recalls = []
    for i in tqdm(range(NUM_RUNS)):
        precision, recall = compute_prd_from_embedding(data_real, data_fake, num_clusters=num_clusters, num_runs=num_runs)
        precisions.append(precision)
        recalls.append(recall)
    return precisions, recalls

In [0]:
precisions, recalls = calc_pr_rec(data_real, data_fake, num_clusters=100, num_runs=20)

In [0]:
pr_aucs = plot_pr_aucs(precisions, recalls)
plt.title('Num_clusters={}, num_runs={}, first third'.format(100, 20))
plt.show()

In [0]:
pr_aucs

## Physical metrics

In [0]:
import matplotlib.lines as mlines
def newline(p1, p2):
    ax = plt.gca()
    xmin, xmax = ax.get_xbound()

    if(p2[0] == p1[0]):
        xmin = xmax = p1[0]
        ymin, ymax = ax.get_ybound()
    else:
        ymax = p1[1]+(p2[1]-p1[1])/(p2[0]-p1[0])*(xmax-p1[0])
        ymin = p1[1]+(p2[1]-p1[1])/(p2[0]-p1[0])*(xmin-p1[0])

    l = mlines.Line2D([xmin,xmax], [ymin,ymax])
    ax.add_line(l)
    return l

def plot_axes_for_shower(ecal, point, p):
    x = np.linspace(-14.5, 14.5, 30)
    y = np.linspace(-14.5, 14.5, 30)

    xx, yy = np.meshgrid(x, y)
    zoff = 25.
    ipic = 3
    orth = np.array([-p[1], p[0]])

    pref = point[:2] + p[:2] * zoff / p[2]

    p1 = pref - 10 * p[:2]
    p2 = pref + 10 * p[:2]
    p3 = pref - 10 * orth
    p4 = pref + 10 * orth

    plt.contourf(xx, yy, np.log(ecal + 1), cmap=plt.cm.inferno)
    newline(p1, p2)
    newline(p3, p4)
    plt.colorbar()

In [0]:
idx = 2
plot_axes_for_shower(EnergyDeposit[idx], point=ParticlePoint[idx].detach().numpy(),
                     p=ParticleMomentum[idx].detach().numpy())

In [0]:
from calogan_metrics import get_assymetry, get_shower_width, get_sparsity_level

In [0]:
assym = get_assymetry(EnergyDeposit, ParticleMomentum.detach().numpy(), ParticlePoint.detach().numpy(), orthog=False)
assym_ortho = get_assymetry(EnergyDeposit, ParticleMomentum.detach().numpy(), ParticlePoint.detach().numpy(), orthog=True)
sh_width = get_shower_width(EnergyDeposit, ParticleMomentum.detach().numpy(), ParticlePoint.detach().numpy(), orthog=False)
sh_width_ortho = get_shower_width(EnergyDeposit, ParticleMomentum.detach().numpy(), ParticlePoint.detach().numpy(), orthog=True)
sparsity_level = get_sparsity_level(EnergyDeposit)

## Longitudual cluster asymmetry

In [0]:
sns.set(font_scale=2)
plt.hist(assym, bins=50, range=[-1, 1], color='red', alpha=0.3, normed=True, label='MC');
plt.xlabel('Longitudual cluster asymmetry')
plt.legend(loc='best')

## Transverse cluster asymmetry

In [0]:
sns.set(font_scale=2)
plt.hist(assym_ortho, bins=50, range=[-1, 1], color='red', alpha=0.3, normed=True, label='MC');
plt.xlabel('Transverse cluster asymmetry')
plt.legend(loc='best')

## Cluster longitudual width

In [0]:
plt.hist(sh_width, bins=50, range=[0, 15], normed=True, alpha=0.3, color='red', label='MC');
plt.title('Shower longitudial width')
plt.legend(loc='best')
plt.xlabel('Cluster longitudual width [cm]')
plt.ylabel('Arbitrary units')

## Cluster trasverse width

In [0]:
plt.hist(sh_width_ortho, bins=50, range=[0,10], normed=True, alpha=0.3, color='blue', label='MC');
#plt.title('Shower transverse width')
plt.legend(loc='best')
plt.xlabel('Cluster trasverse width [cm]')
plt.ylabel('Arbitrary units')

## Sparsity

In [0]:
alphas = np.log(np.logspace(-5, -1, 20))
means_r = np.mean(sparsity_level, axis=1)
stddev_r = np.std(sparsity_level, axis=1)

In [0]:
plt.plot(alphas, means_r, color='red')
plt.fill_between(alphas, means_r-stddev_r, means_r+stddev_r, color='red', alpha=0.3)
plt.legend(['MC'])
plt.title('Sparsity')
plt.xlabel('log10(Threshold/GeV)')
plt.ylabel('Fraction of cells above threshold')

In [0]:
from calogan_metrics import get_physical_stats
real_phys_stats = get_physical_stats(EnergyDeposit, ParticleMomentum.detach().numpy(), ParticlePoint.detach().numpy())
gen_phys_stats = get_physical_stats(EnergyDeposit_gen, ParticleMomentum.detach().numpy(), ParticlePoint.detach().numpy())

In [0]:
precisions, recalls = calc_pr_rec(real_phys_stats, gen_phys_stats, num_clusters=100, num_runs=20)

In [0]:
pr_aucs = plot_pr_aucs(precisions, recalls)
plt.title('Num_clusters={}, num_runs={}, first third'.format(100, 20))
plt.show()

In [0]:
pr_aucs